In [23]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle.json': b'{"username":"norihisa","key":"d016ea67c3249f44d2090f6c16a2ade1"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [3]:
!pip install kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c digit-recognizer

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!pip install -q keras


In [7]:
import numpy as np 
import pandas as pd 

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [0]:
# load training & test datasets
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")


In [0]:
# pandas to numpy
y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

del train

# normalize
X_train = X_train/255.0
test = test/255.0

# reshape the data so that the data
# represents (label, img_rows, img_cols, grayscale)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# one-hot vector as a label
y_train = to_categorical(y_train, num_classes=10)


In [10]:
# convolution -> batch normalization -> ReLU actuvation -> pooling
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

W0715 01:24:17.440569 140574059636608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 01:24:17.459267 140574059636608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 01:24:17.462018 140574059636608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 01:24:17.487982 140574059636608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0715 01:24:17.488728 1405740596

In [11]:
# compile model
optimizer = RMSprop(lr=0.001, 
                    rho=0.9, 
                    epsilon=1e-08, 
                    decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


W0715 01:24:18.549490 140574059636608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# cross validation
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=0.10, 
                                                  random_state=1220)

In [0]:
# data argumentation
gen = ImageDataGenerator(rotation_range=8, 
                         width_shift_range=0.08, 
                         shear_range=0.3,
                         height_shift_range=0.08, 
                         zoom_range=0.08)
train_generator = gen.flow(X_train, y_train, batch_size=64)

In [0]:
# learning rate
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)


In [15]:
# model training
model.fit_generator(train_generator, 
                    epochs=30, 
                    validation_data = (X_val, y_val), 
                    verbose=2, 
                    steps_per_epoch=X_train.shape[0]/36,
                    callbacks=[learning_rate_reduction])


W0715 01:24:19.126245 140574059636608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
 - 21s - loss: 0.1077 - acc: 0.9663 - val_loss: 0.0836 - val_acc: 0.9769
Epoch 2/30
 - 19s - loss: 0.0468 - acc: 0.9849 - val_loss: 0.0464 - val_acc: 0.9874
Epoch 3/30
 - 19s - loss: 0.0386 - acc: 0.9884 - val_loss: 0.0565 - val_acc: 0.9845
Epoch 4/30
 - 18s - loss: 0.0343 - acc: 0.9897 - val_loss: 0.0387 - val_acc: 0.9907
Epoch 5/30
 - 19s - loss: 0.0306 - acc: 0.9915 - val_loss: 0.0352 - val_acc: 0.9914
Epoch 6/30
 - 19s - loss: 0.0281 - acc: 0.9921 - val_loss: 0.0551 - val_acc: 0.9910
Epoch 7/30
 - 19s - loss: 0.0249 - acc: 0.9928 - val_loss: 0.0670 - val_acc: 0.9874
Epoch 8/30
 - 18s - loss: 0.0236 - acc: 0.9933 - val_loss: 0.0715 - val_acc: 0.9862

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 9/30
 - 18s - loss: 0.0164 - acc: 0.9953 - val_loss: 0.0425 - val_acc: 0.9926
Epoch 10/30
 - 18s - loss: 0.0162 - acc: 0.9956 - val_loss: 0.0582 - val_acc: 0.9919
Epoch 11/30
 - 19s - loss: 0.0140 - acc: 0.9962 - val_loss: 0.0509 - val_acc: 

In [0]:
# model prediction on test data
predictions = model.predict_classes(test, verbose=0)


In [0]:
# make a submission file
submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
    "Label": predictions})
submissions.to_csv("my_submission.csv", index=False, header=True)


In [24]:
!ls my_submission.csv

my_submission.csv


In [25]:
# submit the file to kaggle
!kaggle competitions submit digit-recognizer -f my_submission.csv -m "Yeah! I submit my file through the Google Colab!"


100% 208k/208k [00:03<00:00, 58.8kB/s]
403 - You cannot submit until you accept the competition rules
